In [150]:
import pandas as pd
import plotly.express as px
import re
from datetime import datetime, timedelta, fromtimestamp
import numpy as np

In [151]:
df = pd.read_csv('idsc_test.csv')

In [152]:
rota = df['snapshot_radar'].head(1).values[0]

# Usando expressão regular para encontrar as coordenadas
coordenadas = re.findall(r'-?\d+\.\d+', rota)

# Dividir as coordenadas em pares de longitude e latitude
pares = [(float(coordenadas[i]), float(coordenadas[i+1])) for i in range(0, len(coordenadas), 2)]

map = pd.DataFrame(pares, columns=['Latitude', 'Longitude'])

In [158]:
df.head(1).T

,0
flightid,7945735584a3297121c4f5ae0de8ecd1
origem,SBKP
destino,SBSV
dt_dep,1685446563
hora_ref,1685444400
snapshot_radar,MULTIPOINT ((-0.8807200448127578 -0.5148453155...
path,http://satelite.cptec.inpe.br/repositoriogoes/...
hora_esperas,1685440800
esperas,0
aero_esperas,SBSV


In [155]:
colunas = ['dt_dep', 'hora_ref', 'hora_esperas', 'hora_metar', 'hora_metaf', 'hora_tcp']
for name in colunas:
    df[name] = pd.to_datetime(df[name])

In [156]:
null_indices = df.index[df['hora_metaf'].isna()]

df.loc[null_indices, 'hora_metaf'] = df.loc[null_indices, 'hora_metar'] + timedelta(hours=1)
df.loc[null_indices, 'metaf'] = df.loc[null_indices, 'metar']
df.loc[null_indices, 'aero_metaf'] = df.loc[null_indices, 'aero_metar']

In [157]:
for name in colunas:
    df[name] = df[name].apply(lambda x : x.timestamp()).astype(int)

In [141]:
# df.apply(lambda row: row['hora_metar'] + timedelta(days=1) if pd.isna(row['hora_metaf']) else row['hora_metaf'], axis=1)

In [214]:
def convert(timestamp):
    data_hora = datetime.datetime.fromtimestamp(timestamp/1000)
    return data_hora.strftime('%d/%m/%Y %H:%M:%S')

In [229]:
bimtra = pd.read_csv('Dados/bimtra.csv')

In [226]:
bimtra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300347 entries, 0 to 300346
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   flightid  300347 non-null  object
 1   origem    300347 non-null  object
 2   destino   300347 non-null  object
 3   dt_dep    300347 non-null  int64 
 4   dt_arr    300347 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.5+ MB


In [230]:
bimtra['dt_dep'] = bimtra['dt_dep'].apply(convert)
bimtra['dt_arr'] = bimtra['dt_arr'].apply(convert)

In [231]:
bimtra['dt_dep'] = pd.to_datetime(bimtra['dt_dep'],format='%d/%m/%Y %H:%M:%S')
bimtra['dt_arr'] = pd.to_datetime(bimtra['dt_arr'],format='%d/%m/%Y %H:%M:%S')

In [232]:
bimtra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300347 entries, 0 to 300346
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   flightid  300347 non-null  object        
 1   origem    300347 non-null  object        
 2   destino   300347 non-null  object        
 3   dt_dep    300347 non-null  datetime64[ns]
 4   dt_arr    300347 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 11.5+ MB


In [234]:
bimtra['hora_ref'] = bimtra['dt_dep'].dt.floor('H')

In [235]:
bimtra

,flightid,origem,destino,dt_dep,dt_arr,hora_ref
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-05-31 21:44:57,2022-05-31 22:26:00,2022-05-31 21:00:00
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-05-31 21:57:01,2022-05-31 22:32:53,2022-05-31 21:00:00
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-05-31 21:26:52,2022-05-31 22:43:45,2022-05-31 21:00:00
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-05-31 21:58:40,2022-05-31 22:41:41,2022-05-31 21:00:00
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-05-31 22:00:38,2022-05-31 23:33:05,2022-05-31 22:00:00
...,...,...,...,...,...,...
300342,c4321a6a09b76969df53965fb5390f12,SBBR,SBRF,2023-05-13 09:14:19,2023-05-13 11:20:23,2023-05-13 09:00:00
300343,e4f5686cce45083566945c8946e5d120,SBBR,SBGR,2023-05-13 06:33:15,2023-05-13 07:58:41,2023-05-13 06:00:00
300344,04f65a7c44cf176215520b4c9b4eb37f,SBCF,SBRF,2023-05-13 13:53:53,2023-05-13 16:00:24,2023-05-13 13:00:00
300345,f72a0718e581d01c695faeeff35d81f0,SBSV,SBGR,2023-05-13 02:28:58,2023-05-13 04:54:41,2023-05-13 02:00:00


In [237]:
from metar import Metar
obs = Metar.Metar('METAR KEWR 111851Z VRB03G19KT 2SM R04R/3000VP6000FT TSRA BR FEW015 BKN040CB BKN065 OVC200 22/22 A2987 RMK AO2 PK WND 29028/1817 WSHFT 1812 TSB05RAB22 SLP114 FRQ LTGICCCCG TS OHD AND NW -N-E MOV NE P0013 T02270215')
print (obs.string())

station: KEWR
type: routine report, cycle 19 (automatic report)
time: Mon Sep 11 18:51:00 2023
temperature: 22.7 C
dew point: 21.5 C
wind: variable at 3 knots, gusting to 19 knots
peak wind: WNW at 28 knots at 18:17
wind shift: 18:12
visibility: 2 miles
visual range: on runway 04R, from 3000 to greater than 6000 feet
pressure: 1011.5 mb
weather: thunderstorm with rain; mist
sky: a few clouds at 1500 feet
     broken cumulonimbus at 4000 feet
     broken clouds at 6500 feet
     overcast at 20000 feet
sea-level pressure: 1011.4 mb
1-hour precipitation: 0.13in
remarks:
- Automated station (type 2)
- peak wind 28kt from 290 degrees at 18:17
- wind shift at 18:12
- frequent lightning (intracloud,cloud-to-cloud,cloud-to-ground)
- thunderstorm overhead and NW
- TSB05RAB22 -N-E MOV NE
METAR: METAR KEWR 111851Z VRB03G19KT 2SM R04R/3000VP6000FT TSRA BR FEW015 BKN040CB BKN065 OVC200 22/22 A2987 RMK AO2 PK WND 29028/1817 WSHFT 1812 TSB05RAB22 SLP114 FRQ LTGICCCCG TS OHD AND NW -N-E MOV NE P0013 T